### Превращение «черного» ящика в «белый ящик»: пути решений

При рассмотрении дерева решений интуитивно понятно, что для каждого решения, принимаемого деревом (или лесом), имеется путь (или пути) от корня дерева до листа. Путь состоит из серии решений, полученных с помощью конкретного признака, каждый из которых вносит свой вклад в итоговый прогноз.
Дерево решений с $M$ листьями делит пространство на $M$ областей $R_m$, 1 ≤ $m$ ≤ $M$. В классическом определении (например,  в классическом труде «Elements of Statistical Learning») прогнозная функция дерева определяется так:

$\Large F(x) = \sum_{M=1}^m C_mI(x,R_m)$

где:

$M$ - количество листьев в дереве (т.е. области в пространстве признаков);

$R_m$ - область в пространстве признаков (соответствующая листу $m$);

$c_m$ - константа, соответствующая области $m$;

$I$ - индикаторная функция (возвращающая 1, если $x$ $\in$ $R_m$, 0 в противном случае). 

Значение $c_m$ определяется на этапе обучения дерева и в случае деревьев регрессии соответствует среднему значению зависимой переменной в выборке наблюдений, относящейся к области $R_m$ (в случае дерева классификации значение cm соответствует пропорциям классов зависимой переменной). Определение краткое и фиксирует содержательный смысл дерева: решающая функция возвращает значение в правильном листе дерева. Однако это определение игнорирует наличие путей, проходящих через узлы-решения, и информацию, содержащуюся в них.


### Пример: данные о ценах на жилье в Бостоне

Давайте возьмем набор данных о ценах на жилье в Бостоне, который включает цены на жилье в пригородах Бостона, а также ряд ключевых признаков, таких как качество воздуха (переменная `NOX`), расстояние до центра города (`DIS`) и ряд других. Мы построим дерево регрессии (глубина 3 взята для удобства чтения), чтобы предсказать цены на жилье. Как обычно, у дерева есть условия для каждого внутреннего узла и значение, связанное с каждым листом (то есть спрогнозированное значение). Но дополнительно мы выведем значение в каждом внутреннем узле, то есть среднее значение зависимой переменной в этой области.


<img src='img/picture.png'>


Ниже вы увидите прогноз, записанный с точки зрения изменений среднего значения зависимой переменной вдоль пути прогнозирования (выделен на рисунке красным), вместе с названиями признаков, которые обусловили изменение среднего значения (числа являются приблизительными из-за округления).


<img src='img/picture2.png'>


Этот пример показывает, что существует еще один, более компактный способ вычисления прогнозов, а именно вычисление прогноза с помощью последовательности областей, соответствующих каждому узлу/решению дерева. Поскольку каждое решение принимается функцией и решение либо увеличивает, либо уменьшает значение зависимой переменной, начиная с родительского узла, то прогноз может быть определен как сумма вкладов признаков плюс «смещение» (то есть среднее значение зависимой переменной в корневом узле, который охватывает весь обучающий набор).

Прогнозную функцию можно записать как

$\Large F(x) = c + \sum_{k=1}^K contrib(x,k)$

где

$K$ - количество признаков;

$c$ - значение зависимой переменной в корневом узле;

$contrib(x,k)$ - вклад $k$-го признака в векторе признаков $x$.

Это немного похоже на формулу линейной регрессии  $f(x) = a + bx$. Для линейной регрессии коэффициенты $b$ фиксированы, каждый признак имеет определенный вес, который определяет вклад признака. Для дерева решений вклад каждого признака не является конкретным определенным значением, однако зависит от остальных признаков, определяющих путь решения, который проходит вдоль дерева и, следовательно, учитывает вклады признаков, которые встречаются на пути.


### От деревьев решений к случайному лесу

Итак, каким образом мы можем перейти от дерева решений к случайному лесу? Это довольно просто, поскольку прогнозом леса является усредненный прогноз его деревьев:

$\Large F(x) = \frac{1}{J} \sum_{j=1}^J f_i(x)$

где

$J$ - это количество деревьев в лесу.

Исходя из этой формулы, легко увидеть, что для леса прогноз – это просто усредненное смещение плюс усредненный вклад каждого признака:

$\Large F(x) = \frac{1}{J}\sum_{j=1}^J c_j + \sum_{k=1}^K(\frac{1}{J} \sum_{j=1}^Jcontrib_j(x,k))$

$c_j$ - значение зависимой переменной в корневом узле $j$-ного дерева леса.

Алгоритм интерпретации случайного леса реализован в питоновском пакете `treeinterpreter` (устанавливается с помощью команды `pip install treeinterpreter`), который выполняет декомпозицию прогнозов, полученных с помощью дерева решений или случайного леса.

In [1]:
# загружаем необходимые библиотеки и классы (treeinterpreter можно 
# установить с помощью команды pip install treeinterpreter)
from treeinterpreter import treeinterpreter as ti
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

In [2]:
# загружаем данные для задачи регрессии
from sklearn.datasets import load_boston
boston = load_boston()
# Подгоняем ансамбль деревьев регрессии
rf = RandomForestRegressor(random_state = 42)
rf.fit(boston.data, boston.target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [3]:
# отдельно запишем наблюдение, для которого будет смотреть прогноз
observation=boston.data[300:301]

In [4]:
# печатаем прогноз для наблюдения
print("Прогноз для наблюдения:", rf.predict(observation))

Прогноз для наблюдения: [26.36]


In [5]:
# выполним декомпозицию прогноза для наблюдения
prediction, bias, contributions = ti.predict(rf, observation)
# Печатаем результаты декомпозиции прогноза
print("Прогноз", prediction)
print("Смещение (среднее зависимой переменной в обучающем наборе)", bias)
print("Вклады признаков:")
for c, feature in zip(contributions[0], boston.feature_names):
    print(feature, c)

Прогноз [26.36]
Смещение (среднее зависимой переменной в обучающем наборе) [22.63029644]
Вклады признаков:
CRIM -0.3360322270322268
ZN -0.03758974358974321
INDUS 0.21589115418894772
CHAS 0.0
NOX 0.2513173222876087
RM 2.9133520427656747
AGE -0.6413400809716603
DIS -1.576270071161994
RAD -0.041999999999999815
TAX -0.30116486066193315
PTRATIO 0.0018204192462260237
B -0.03514957264957275
LSTAT 3.3168691748909183


In [6]:
# печатаем результаты декомпозиции прогноза, отсортировав 
# вклады признаков по убыванию абсолютного значения и 
# выполнив округление до второго знака
print("Смещение (среднее зависимой переменной в обучающем наборе)", bias)
print("Вклады признаков:")
for c, feature in sorted(zip(contributions[0], 
                             boston.feature_names), 
                             key = lambda x: -abs(x[0])):
    print(feature, round(c, 2))    

Смещение (среднее зависимой переменной в обучающем наборе) [22.63029644]
Вклады признаков:
LSTAT 3.32
RM 2.91
DIS -1.58
AGE -0.64
CRIM -0.34
TAX -0.3
NOX 0.25
INDUS 0.22
RAD -0.04
ZN -0.04
B -0.04
PTRATIO 0.0
CHAS 0.0


Теперь вклады отсортированы по уменьшению абсолютных значений вкладов. Здесь мы видим, что наибольшим положительным вкладом обладают предикторы `LSTAT` и `RM`, то есть они в наибольшей степени увеличивают прогнозное значение зависимой переменной для данного наблюдения. При этом для другого взятого наблюдения может быть обратная ситуация: эти предикторы могут значительно уменьшать прогнозное значение зависимой переменной. Таким образом, между вкладами признаков нельзя проводить аналогию с регрессионными коэффициентами, которые будут для всех наблюдений одинаковыми. Нельзя сказать, что всегда с возрастанием значений переменных `LSTAT` и `RM` будет увеличиваться цена на жилье. Если дальше смотреть на вывод, то видно, что наибольший отрицательный вклад дает переменная `DIS`. Опять же помним, что для другого наблюдения эта переменная может дать положительный вклад. 

Давайте проверим, корректна ли декомпозиция прогноза. Для этого мы должны сложить смещение и вклады признаков и получить в сумме спрогнозированное значение.

In [7]:
# сложим смещение и вклады признаков, чтобы проверить,
# совпадает ли полученный результат с прогнозом
print(prediction)
print(bias + np.sum(contributions, axis = 1))

[26.36]
[26.36]


Результат сложения совпадает со спрогнозированным значением. Теперь загрузим данные о заемщиках банка для решения задачи классификации и построим ансамбль деревьев классификации.

In [8]:
# загружаем данные для задачи классификации в пандасовский DataFrame
data = pd.read_csv("C:/Trees/BankloanPy.csv", encoding = "cp1251", sep = ";")
# Формируем массив признаков и массив меток
y = data.loc[:, "default"]
X = data.loc[:, "age":"othdebt"]
# записываем названия признаков, чтобы потом сопоставить их значениям вкладов
features_names = X.columns
# разбиваем на обучение и тест
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)
# подгоняем ансамбль деревьев классификации
forest=RandomForestClassifier(n_estimators = 200, random_state = 42)
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [9]:
# печатаем результаты декомпозиции прогноза для конкретного наблюдения
prediction_test, bias_test, contributions_test = ti.predict(forest, X_test[100:101])
print("Прогноз", prediction_test)
print("Смещения (вероятности классов зависимой переменной в обучающем наборе)", bias_test)
print("Вклады признаков:")
for c, feature in zip(contributions_test[0], features_names):
    print(feature, c)

Прогноз [[0.95 0.05]]
Смещения (вероятности классов зависимой переменной в обучающем наборе) [[0.64397143 0.35602857]]
Вклады признаков:
age [ 0.04751199 -0.04751199]
employ [ 0.07158361 -0.07158361]
address [ 0.0427569 -0.0427569]
income [ 0.04874334 -0.04874334]
debtinc [ 0.07574518 -0.07574518]
creddebt [ 0.00998228 -0.00998228]
othdebt [ 0.00970526 -0.00970526]


Из вывода видно, что мы получаем значения смещения и вклады признаков по двум классам зависимой переменной. Значение смещения в каждом классе – это вероятность соответствующего класса зависимой переменной в выборке. Вклады признаков в каждом классе  показывают, в какой степени те или иные признаки увеличивают или уменьшают вероятность соответствующего класса. Значения вкладов конкретного признака по классам зеркальны, что логично: переменная `age` на 0,05 увеличивает вероятность отрицательного класса (вероятность того, что заемщик окажется «хорошим)», что эквивалентно уменьшению вероятности положительного класса (вероятности того, что заемщик окажется «плохим») на 0,05. И вновь вспомним, что такая картина верна только для данного наблюдения.